# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902032


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:28,  2.95s/it]

Rendering models:   9%|▉         | 3/32 [00:06<01:02,  2.14s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<01:04,  2.30s/it]

Rendering models:  16%|█▌        | 5/32 [00:10<00:53,  2.00s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:25<00:01,  1.58it/s]

Rendering models:  94%|█████████▍| 30/32 [00:25<00:01,  1.70it/s]

Rendering models:  97%|█████████▋| 31/32 [00:27<00:00,  1.31it/s]

Rendering models: 100%|██████████| 32/32 [00:27<00:00,  1.29it/s]

equidad1                               0.020132
kayleebug2017                          0.000640
CTidwell3                              0.000222
tingard                                0.000174
Betelgeuse12                           0.000253
not-logged-in-87ebe289c5b4fb76a06c     0.733145
nickoftona                             0.000337
mollollo                               0.000192
ElisabethB                             0.000205
not-logged-in-75af55cf19be29c510aa     0.019994
not-logged-in-8c0c44faf754d344b18a     0.014581
astro_lab_ncmns                        0.000618
astro_lab_ncmns                        0.000625
not-logged-in-258b74099ecf199aff36     2.827471
not-logged-in-d6cc15fe4c5aab399c3f     0.000251
pangeli5                               0.002136
clairedeu                              0.000274
gabrielweiss                           0.002932
Planetme                               0.000380
not-logged-in-214814e52004b42491a4     0.093612
Saharisunshine                         0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())